# Documentation: Data base managers

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from astropy.coordinates import Angle
import astropy.units as u
import os
import platform

import constraints as c
from db import DBCreator, TelescopeManager, FieldManager, GuidestarManager, ObservationManager
from fieldgrid import FieldGridIsoLat

## `DBCreator`: create survey planner database

The `DBCreator` class provides the basic functionalities to create a new database:

In [53]:
db_name = 'test_planner_init.sqlite3'
db = DBCreator(db_name)

### Create database

In [40]:
created = db.create()

Database file exists. Overwrite (y) or cancel (enter)? y


Database 'test_planner_init.sqlite3' created.
Table 'Fields' created.
Table 'Telescopes' created.
Table 'ParameterSets' created.
Table 'Constraints' created.
Table 'Parameters' created.
Table 'ParameterNames' created.
Table 'Observability' created.
Table 'ObservabilityStatus' created.
Table 'ObsWindows' created.
Table 'TimeRanges' created.
Table 'Observations' created.
Table 'Guidestars' created.
Table 'Filters' created.
Constraints added to table 'Constraints'.
Statuses added to table 'ObservabilityStatus'.
Database creation finished.

Note: Next you need to add observatories, constraints, fields, guidestars, and observations.


### Add telescopes

In [41]:
if created:
    name = 'Skinakas'
    lat = Angle('35:12:43 deg')
    lon = Angle('24:53:57 deg')
    height = 1750.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'Skinakas' added.


In [42]:
if created:
    name = 'SAAO'
    lat = Angle('-32:22:46 deg')
    lon = Angle('20:48:38.5 deg')
    height = 1798.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'SAAO' added.


It is not possible to add two telescopes with the same name:

In [43]:
if created:
    name = 'SAAO'
    lat = Angle('-32:22:46 deg')
    lon = Angle('20:48:38.5 deg')
    height = 1798.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'SAAO' already exists. Name needs to be unique.


### Add constraints

We add constraints to each telescope separately. Some constraints are the same, some are telescope specific.

In [44]:
if created:
    # general limits:
    twilight = 'nautical'
    airmass_limit = c.AirmassLimit(2.)
    moon_distance = c.MoonDistance(10.)
    
    # Skinakas specific limits:
    hourangle_limit = c.HourangleLimit(5.33)
    
    # SAAO specific limits:
    polygon_ha = [-4, -4, 0.8, 0.8, 1.7, 2.4, 3.2, 2.8]
    polygon_dec = [0.1, -52, -65, -85, -74.5, -70.0, -60, 0.1]
    ha_dec_limit = c.PolyHADecLimit(polygon_ha, polygon_dec)
    
    db.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))
    db.add_constraints('SAAO', twilight, constraints=(airmass_limit, moon_distance, ha_dec_limit))

Constraint 'AirmassLimit' for telescope 'Skinakas'.
Constraint 'HourangleLimit' for telescope 'Skinakas'.
Constraint 'MoonDistance' for telescope 'Skinakas'.
Constraint 'AirmassLimit' for telescope 'SAAO'.
Constraint 'MoonDistance' for telescope 'SAAO'.
Constraint 'PolyHADecLimit' for telescope 'SAAO'.


It is not possible to add constraints to a non-existing telescope. Thus, telescopes need to be added first.

In [45]:
db.add_constraints('does-not-exist', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))

Telescope 'does-not-exist' does not exist in database. Use TelescopeManager() to manage telescopes or add new ones.


### Add fields

First, we create the field grid using the `FieldGridIsoLat` class. For a quick demonstration we do not create the actual Pasiphae grid, but a coarser grid with fewer fields.

Add Northern fields:

In [48]:
fov = Angle(5 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(90 * u.deg)
dec_lim_south = Angle(0 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 1097


In [50]:
if created:
    db.add_fields(grid, 'Skinakas', active=True)

1097 fields added to database.                   


Add Southern fields:

In [51]:
fov = Angle(7 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(0 * u.deg)
dec_lim_south = Angle(-90 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 547


In [52]:
if created:
    db.add_fields(grid, 'SAAO', active=True)

547 fields added to database.                   


### Add guidestars

For a simple demonstration we claim there is a guide star at each field center, i.e. we use the field centers as coordinates for the guide stars.

In [59]:
# get fields:
manager = FieldManager(db_name)
field_ids = []
field_center_ras = []
field_center_decs = []

for field in manager.get_fields():
    field_ids.append(field[0])
    field_center_ras.append(field[2])
    field_center_decs.append(field[3])

We are going to add guidestars for all but the last to fields. At the end we will be warned that there are fields without associated guidestars.

In [61]:
db.add_guidestars(field_ids[:-2], field_center_ras[:-2], field_center_decs[:-2], warn_missing=True)

1642 new guidestars added to database.
Fields with the following IDs do not have any guidestars associated:
1643, 1644


By setting `warn_rep`, a lower limit that guidestars for the same field need to be separated, we can also get a warning when we try to add a new guidestar that is considered a duplicate of one already stored:

In [63]:
db.add_guidestars(field_ids[0], field_center_ras[0], field_center_decs[0], warn_rep=Angle(2*u.arcmin), warn_missing=False)

New guidestar
  field ID: 1
  RA:   00d00m00s
  Dec: +02d30m00s
is close to the following stored guidestar(s):
1. Guidestar ID: 1
  RA:   00d00m00s
   Dec: +02d30m00s
   separation: 0.0000 deg


Add this new guidestar anyway? (y/n)  n


0 new guidestars added to database.


As a final check, we can also set `warn_sep`, a maximum limit by which the guidestar may be separated from the corresponding field center. We get a warning if a guidestart is too far off:

In [64]:
db.add_guidestars(
        field_ids[-1], field_center_ras[-1]+Angle(18*u.arcmin).rad, field_center_decs[-1],
        warn_sep=Angle(15*u.arcmin), warn_missing=False)

New guide star 0 for field ID 1644 is too far from the field center with separation 0d17m58.51989416s.


Add it to the database anyway? (y/n)  n


0 new guidestars added to database.


### Add observations

We can simply add the same observation to each field. The filter does not exist in the new database. We are automatically asked, whether or not to add it.

In [66]:
db.add_observations(30., 2, 'r')

Filter 'r does not exist. Add it to data base? (y/n) y


1644 observations added to data base.


If at least one observation with the same parameters exist for a field and has not been finished yet, we automatically get notified that we are trying to add a duplicate:

In [67]:
db.add_observations(30., 2, 'r')

1 observation(s) with the same parameters already exist in data base. 0 out of those are finished. Add new observation anyway? (y/n, 'ALL' to add all following without asking, or 'NONE' to skip all existing observations that have not been finished). NONE


0 observations added to data base.


### Database info

<div class="alert alert-block alert-warning">

- Add dbinfo() method to DBManager that prints out general information about the database. Include it here in the documentation.

</div>

## Database managers

**Important:** Any changes to the database should be made via these managers. The database should not be edited manually, because that may severly interfere with the survey planning.

For a demonstration of the managers we copy the database that was created above and work with the copy in the following.

In [3]:
db_init = 'test_planner_init.sqlite3'
db_name = 'test_planner_temp.sqlite3'

if platform.system() == 'Linux':
    os.system(f'cp {db_init} {db_name}')
elif platform.system() == 'Windows':
    os.system(f'xcopy {db_init} {db_name} /y')
else:
    raise ValueError('Unknown operating system.')

### `TelescopeManager`

The `TelescopeManager` class provides the methods to add telescopes and constraints to the database and read them from the database.

#### Add telescope

In [4]:
manager = TelescopeManager(db_name)

We can use the same method as in the `DBCreator` class to add a telescope. Duplications are not allowed:

In [5]:
name = 'Skinakas'
lat = Angle('35:12:43 deg')
lon = Angle('24:53:57 deg')
height = 1750.
utc_offset = 2.
manager.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'Skinakas' already exists. Name needs to be unique.


**Note:** There is not way to change a telescope. Changing its parameters would require that all observabilities need to be re-calculated. Therefore, it makes sense to start with a new database.

#### Add/change constraints

We can add constraints as we did with the `DBCreator` class. If the telescope already has constraints associated we get a warning:

In [6]:
# general limits:
twilight = 'nautical'
airmass_limit = c.AirmassLimit(2.)
moon_distance = c.MoonDistance(10.)

# Skinakas specific limits:
hourangle_limit = c.HourangleLimit(5.33)

manager.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))

**Note:** The method does not check whether or not these new constraints differ in any way from the already stored ones. Above we try to add the exact same constraints we already added. This would be unwise to do, because it would invalidate all observabilities that were already calculated, even though they would technically still be valid as the constraints are the same.

**Note:** Adding a new set of constraints, means that we need to define all constraints that should from now one be associated with the telescope. We cannot add a single constraint to the ones existing already in the database. 

Let's say we want to decrease the Moon separation limit:

In [7]:
# general limits:
twilight = 'nautical'
airmass_limit = c.AirmassLimit(2.)
moon_distance = c.MoonDistance(5.)

# Skinakas specific limits:
hourangle_limit = c.HourangleLimit(5.33)

manager.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))

Parameter set with ID 1 deactivated.
0 corresponding observabilities deactivated.
0 corresponding observing windows deactivated.
0 corresponding time ranges deactivated.
Constraint 'AirmassLimit' for telescope 'Skinakas'.
Constraint 'HourangleLimit' for telescope 'Skinakas'.
Constraint 'MoonDistance' for telescope 'Skinakas'.


**Note:** Constraints must always be changed this way. Only this way ensures that the observabilities are calculated anew. This would not be the case with manual changes to the database.

#### Print info about telescopes and constraints

We can use the `info()` method to learn about what telescopes are stored in the database. If we use `constraints=True`, we additionally get info about the currently active, associated constraints. With `constraints='all'` we learn about all associated parameter sets, including the inactive ones:

In [85]:
manager.info(constraints='all')

======== TELECOPES ========
2 telescopes stored in database.
---------------------------
Telescope ID: 1
Name: Skinakas
Latitude :   35.21 deg N
Longitude:   24.90 deg E
Height:    1750.00 m            
---------------------------
2 associated parameter sets
---------------------------
Parameter set ID: 1
Active: False            
Constraints:
* Twilight
  - twilight: -12.0
* AirmassLimit
  - limit: 2.0
  - conversion: secz
* HourangleLimit
  - limit: 5.33
  - limit_lo: -5.33
* MoonDistance
  - limit: 10.0
---------------------------
Parameter set ID: 3
Active: True            
Constraints:
* Twilight
  - twilight: -12.0
* AirmassLimit
  - limit: 2.0
  - conversion: secz
* HourangleLimit
  - limit: 5.33
  - limit_lo: -5.33
* MoonDistance
  - limit: 5.0
---------------------------
Telescope ID: 2
Name: SAAO
Latitude :  -32.38 deg N
Longitude:   20.81 deg E
Height:    1798.00 m            
---------------------------
1 associated parameter sets
---------------------------
Parameter set I

#### Get telescopes

`get_telescopes()` gives us **all stored telescopes** in a list. Each list item is a dict that contains the telescope parameters:

In [43]:
manager.get_telescopes()

[{'telescope_id': 1,
  'name': 'Skinakas',
  'lat': <Quantity 0.61456437 rad>,
  'lon': <Quantity 0.43457244 rad>,
  'height': 1750.0,
  'utc_offset': 2.0},
 {'telescope_id': 2,
  'name': 'SAAO',
  'lat': <Quantity -0.56512792 rad>,
  'lon': <Quantity 0.36321514 rad>,
  'height': 1798.0,
  'utc_offset': 2.0}]

We can **specify a telescope name** to get just that telescope:

In [44]:
manager.get_telescopes(name='Skinakas')

{'telescope_id': 1,
 'name': 'Skinakas',
 'lat': <Quantity 0.61456437 rad>,
 'lon': <Quantity 0.43457244 rad>,
 'height': 1750.0,
 'utc_offset': 2.0}

We can also **include the active constraints** that are associated with the telescope, by setting `constraints=True`:

In [45]:
manager.get_telescopes(name='Skinakas', constraints=True)

{'telescope_id': 1,
 'name': 'Skinakas',
 'lat': <Quantity 0.61456437 rad>,
 'lon': <Quantity 0.43457244 rad>,
 'height': 1750.0,
 'utc_offset': 2.0,
 'constraints': {'Twilight': {'twilight': -12.0},
  'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
  'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
  'MoonDistance': {'limit': 5.0}}}

If we want to see all associated constraints, regardless of whether they are active or not, we use `constraints='all'`:

In [54]:
manager.get_telescopes(name=None, constraints='all')

[{'telescope_id': 1,
  'name': 'Skinakas',
  'lat': <Quantity 0.61456437 rad>,
  'lon': <Quantity 0.43457244 rad>,
  'height': 1750.0,
  'utc_offset': 2.0,
  'parameter_sets': [{'parameter_set_id': 1,
    'telescope': 'Skinakas',
    'active': False,
    'constraints': {'Twilight': {'twilight': -12.0},
     'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
     'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
     'MoonDistance': {'limit': 10.0}}},
   {'parameter_set_id': 2,
    'telescope': 'SAAO',
    'active': True,
    'constraints': {'Twilight': {'twilight': -12.0},
     'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
     'MoonDistance': {'limit': 10.0},
     'PolyHADecLimit': {'ha': [-4.0, -4.0, 0.8, 0.8, 1.7, 2.4, 3.2, 2.8],
      'dec': [0.1, -52.0, -65.0, -85.0, -74.5, -70.0, -60.0, 0.1]}}},
   {'parameter_set_id': 3,
    'telescope': 'Skinakas',
    'active': True,
    'constraints': {'Twilight': {'twilight': -12.0},
     'AirmassLimit': {'limit': 2.0, 'conver

#### Get constraints

In `get_constraints()` we can specify a telescope, if we want to get only the constraints associated with that telescope. With `active=True` we only get active constraints. Otherwise, we get active and inactive constraints. 

If we specify the telescope and set `active=True` there is only one set of constraints, because only one set can be active for a telescope. 
Otherwise, we may get multiple sets as a `list`. Sets contain all the constraints that were added to a telescope via one call of the `add_constraints()` method.

Here, we are calling for **one specific telescope and its active constraints**:

In [30]:
manager.get_constraints(telescope='Skinakas', active=True)

{'parameter_set_id': 3,
 'telescope': 'Skinakas',
 'active': True,
 'constraints': {'Twilight': {'twilight': -12.0},
  'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
  'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
  'MoonDistance': {'limit': 5.0}}}

We can get an overview over **all existing constraints** like this:

In [31]:
manager.get_constraints(telescope=None, active=False)

[{'parameter_set_id': 1,
  'telescope': 'Skinakas',
  'active': False,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
   'MoonDistance': {'limit': 10.0}}},
 {'parameter_set_id': 2,
  'telescope': 'SAAO',
  'active': True,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'MoonDistance': {'limit': 10.0},
   'PolyHADecLimit': {'ha': [-4.0, -4.0, 0.8, 0.8, 1.7, 2.4, 3.2, 2.8],
    'dec': [0.1, -52.0, -65.0, -85.0, -74.5, -70.0, -60.0, 0.1]}}},
 {'parameter_set_id': 3,
  'telescope': 'Skinakas',
  'active': True,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
   'MoonDistance': {'limit': 5.0}}}]

#### Other methods

Other methods are used by the survey planning software.

## `FieldManager`: manage fields

## `GuidestarManager`: manage guide stars

## `ObservationManager`: manage observations